In [2]:
import xarray as xr
import os
import numpy as np

In [69]:
geo_file = 'D:\CSSP\QH/geo_em.d01.nc'
path = 'D:/CSSP/QH/'
nc_file = os.path.join(path,'CSSPout-1981-01-01_23_30_00FSENA.nc')
var_name = 'FSENA'
data = xr.open_dataset(nc_file)[var_name][0,:,:]
out_file = os.path.join(path,'output_data.nc')

# 打开地理文件
geo_dataset = xr.open_dataset(geo_file)
# 获取经纬度数据
x_values = np.array(geo_dataset['XLONG_M'][0,:,:])
y_values = np.array(geo_dataset['XLAT_M'][0,:,:])

data_array = xr.DataArray( 
    data ,
    dims=['y', 'x'],  # 假设的二维空间维度
    coords={
        'lon': (['y', 'x'], y_values),
        'lat': (['y', 'x'], x_values)
    }
)
dataset = xr.Dataset({var_name: data_array})
output_file = 'output_data.nc'
dataset.to_netcdf(output_file)

def create_dataset_with_2d_coords(x_values, y_values, data, var_name, nc_file):
    # 创建xarray的DataArray，x_values和y_values作为坐标
    dims = ('lon', 'lat')
    coords = {
        'xlon': (dims, x_values),
        'xlat': (dims, y_values)
    }
    
    data_array = xr.DataArray(data, coords=coords, dims=dims, name=var_name)
    
    # 创建Dataset
    ds = data_array.to_dataset(name=var_name)
    
    # 设置坐标变量的属性
    ds['lon'].attrs['units'] = 'degrees_east'
    ds['lon'].attrs['standard_name'] = 'longitude'
    ds['lat'].attrs['units'] = 'degrees_north'
    ds['lat'].attrs['standard_name'] = 'latitude'
    
    # 保存到NetCDF文件
    ds.to_netcdf(nc_file)
    print(nc_file)

In [45]:
create_dataset_with_2d_coords(x_values, y_values, data, var_name, nc_file)

D:/CSSP/QH/CSSPout-1981-01-01_23_30_00FSENA1.nc


In [27]:
data = xr.open_dataset(out_file)[var_name][:,:]
data 

<xarray.DataArray 'FSENA' (y: 700, x: 830)>
[581000 values with dtype=float32]
Coordinates:
    lon      (y, x) float32 ...
    lat      (y, x) float32 ...
Dimensions without coordinates: y, x
Attributes:
    long_name:  sensible heat from canopy height to atmosphere
    units:      W/m2

In [67]:
geo_file = 'D:\CSSP\QH/geo_em.d01.nc'
path = 'D:/CSSP/QH/'
mask = 'qh_mask.nc'
out_file = os.path.join(path,'output_data.nc')
mask_file = os.path.join(path,'qh_mask.nc')
ds = xr.open_dataset(geo_file ,engine='netcdf4')
lon = np.array(ds['XLONG_M'])
lat = np.array(ds['XLAT_M'])

geo = np.zeros((700,830,2))

geo[:,:,0] = lon
geo[:,:,1] = lat
#data_all = ds['FSENA'].values

In [33]:
top = 39.9502
bottom = 31.0502
left = 89.05
right = 103.95

lat_index_top = np.unravel_index(np.argmin(np.abs(lat - top)), lat.shape)
lat_index_bottom = np.unravel_index(np.argmin(np.abs(lat - bottom)), lat.shape)
lon_index_right = np.unravel_index(np.argmin(np.abs(lon - right)), lon.shape)
lon_index_left = np.unravel_index(np.argmin(np.abs(lon - left)), lon.shape)

print("Top Latitude Index:", lat_index_top)
print("Bottom Latitude Index:", lat_index_bottom)
print("Right Longitude Index:", lon_index_right)
print("Left Longitude Index:", lon_index_left)

Top Latitude Index: (0, 414, 307)
Bottom Latitude Index: (0, 293, 33)
Right Longitude Index: (0, 127, 448)
Left Longitude Index: (0, 144, 193)


In [41]:




ds = xr.open_dataset(mask_file)
mask = np.array(ds['qh_mask'])
mask.shape

(90, 150)

In [68]:
import numpy as np

# 模拟的二维经度和纬度数据


# 给定的经纬度边界
top = 39.9502
bottom = 31.0502
left = 89.05
right = 103.95

# 计算每个网格点与四个角点的距离平方
distance_squared_top_left = (geo[:,:,0] - left)**2 + (geo[:,:,1] - top)**2
distance_squared_top_right = (geo[:,:,0] - right)**2 + (geo[:,:,1] - top)**2
distance_squared_bottom_left = (geo[:,:,0] - left)**2 + (geo[:,:,1] - bottom)**2
distance_squared_bottom_right = (geo[:,:,0] - right)**2 + (geo[:,:,1] - bottom)**2

# 找到每个角点最小距离的索引
min_index_top_left = np.unravel_index(np.argmin(distance_squared_top_left), distance_squared_top_left.shape)
min_index_top_right = np.unravel_index(np.argmin(distance_squared_top_right), distance_squared_top_right.shape)
min_index_bottom_left = np.unravel_index(np.argmin(distance_squared_bottom_left), distance_squared_bottom_left.shape)
min_index_bottom_right = np.unravel_index(np.argmin(distance_squared_bottom_right), distance_squared_bottom_right.shape)

# 输出每个角点的最近点索引及其经纬度
print("最接近左上角的点的索引:", min_index_top_left)
print("左上角的经度和纬度:", geo[min_index_top_left[0], min_index_top_left[1], 0], geo[min_index_top_left[0], min_index_top_left[1], 1])

print("最接近右上角的点的索引:", min_index_top_right)
print("右上角的经度和纬度:", geo[min_index_top_right[0], min_index_top_right[1], 0], geo[min_index_top_right[0], min_index_top_right[1], 1])

print("最接近左下角的点的索引:", min_index_bottom_left)
print("左下角的经度和纬度:", geo[min_index_bottom_left[0], min_index_bottom_left[1], 0], geo[min_index_bottom_left[0], min_index_bottom_left[1], 1])

print("最接近右下角的点的索引:", min_index_bottom_right)
print("右下角的经度和纬度:", geo[min_index_bottom_right[0], min_index_bottom_right[1], 0], geo[min_index_bottom_right[0], min_index_bottom_right[1], 1])



最接近左上角的点的索引: (422, 231)
左上角的经度和纬度: 89.065185546875 39.94184875488281
最接近右上角的点的索引: (410, 442)
右上角的经度和纬度: 103.93289184570312 39.94832992553711
最接近左下角的点的索引: (259, 209)
左下角的经度和纬度: 89.07302856445312 31.07598876953125
最接近右下角的点的索引: (245, 446)
右下角的经度和纬度: 103.97506713867188 31.051864624023438


In [76]:
import xarray as xr
import numpy as np

# 加载掩码数据集
mask_ds = xr.open_dataset(mask_file)
mask = mask_ds['qh_mask']  # 假设掩码变量名为 'mask_variable'

ds = xr.open_dataset(nc_file)

# 使用 .isel() 方法提取特定的索引范围
# 假设我们处理的是具有两个空间维度的数据集：'y' 和 'x'
subset_ds = ds.isel(xlat=slice(254, 410), xlon=slice(231, 444))

# 插值掩码以匹配目标数据集的网格
interp_mask = mask.interp(
    lat=subset_ds['xlat'].values, 
    lon=subset_ds['xlon'].values,
    method='nearest'  # 使用最近邻插值，因为掩码通常是分类数据
)
output_file_path = 'maskQH.nc'
interp_mask.to_netcdf(output_file_path)

# 查看插值后的掩码
print(interp_mask)


<xarray.DataArray 'qh_mask' (lat: 156, lon: 213)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * lat      (lat) int64 0 1 2 3 4 5 6 7 8 ... 148 149 150 151 152 153 154 155
  * lon      (lon) int64 0 1 2 3 4 5 6 7 8 ... 205 206 207 208 209 210 211 212
Attributes:
    long_name:       qh_mask
    esri_pe_string:  GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1...
    units:           Degree


In [118]:
import xarray as xr
import numpy as np
from scipy.interpolate import RectBivariateSpline

mask_ds = xr.open_dataset(mask_file)
mask = mask_ds['qh_mask']  # 假设掩码变量名为 'mask_variable'
mask = np.array(mask)
# 创建原始掩码数据

original_shape = mask.shape
new_shape = (156, 213)

x = np.linspace(0, 1, original_shape[1])
y = np.linspace(0, 1, original_shape[0])

# 创建新的索引网格
x_new = np.linspace(0, 1, new_shape[1])
y_new = np.linspace(0, 1, new_shape[0])

# 创建插值函数
interpolator = RectBivariateSpline(y, x, mask)

# 执行插值
mask_interpolated = interpolator(y_new, x_new)


# 转换为 xarray.DataArray，你可以添加适当的坐标和维度名
mask_interpolated = np.where(mask_interpolated > 0.95, 1, 0)
mask_da = xr.DataArray(mask_interpolated , dims=['xlat', 'xlon'])



# 创建 Dataset 并添加数据数组
ds = xr.Dataset({'QH_mask': mask_da})

# 保存到 NetCDF 文件
output_file = 'interpolatedQH_mask1.nc'
ds.to_netcdf(output_file)



In [89]:
import xarray as xr 
import os
import numpy as np 

mask_path = 'interpolatedQH_mask.nc'
path = '/data/workspace/swli/Data/CSSPout_forQH/'
tmp_path_base = '/data/workspace/swli/Data/data4QH/'
var_name = 'FSENA'

# 加载掩码数据集
mask_ds = xr.open_dataset(mask_path)
mask = mask_ds['QH_mask'] 
mask = mask == 1

def extract_masked(mask, ncfile, var_name, output_ncfile):
    
 with xr.open_dataset(ncfile) as data :#气象强迫数据
    data = data[var_name][254:410,231:444]#读取数据
    data_var = np.array(data)
    #print(data_PRE.shape)
    #print(mask.shape)
    var_vaild = data_var[mask]                    

    var_vaild.to_netcdf(output_ncfile) 
    print(output_ncfile)   

for year in range(1981, 2023):
    year_path = os.path.join(path, str(year))
    if not os.path.exists(year_path):
        continue
    for file_name in os.listdir(year_path):
        nc_file = os.path.join(year_path, file_name)
        tmp_path = os.path.join(tmp_path_base, str(year), file_name)  # 更新文件路径
        if len(file_name) == 35:
                extract_masked(mask_path, nc_file,var_name,tmp_path)
                

In [113]:
geo_file = 'D:\CSSP\QH/geo_em.d01.nc'
path = 'D:/CSSP/QH/'
nc_file = os.path.join(path,'CSSPout-1981-01-01_23_30_00FSENA.nc')
var_name = 'FSENA'
mask_path = 'interpolatedQH_mask1.nc'
var_vaild = np.zeros((156,213))
# 加载掩码数据集并创建掩码
mask_ds = xr.open_dataset(mask_path)
mask = mask_ds['QH_mask'] == 1  # 创建布尔掩码

# 加载数据并选择特定范围
data_ds = xr.open_dataset(nc_file)
data_subset = data_ds[var_name].isel(time=0, xlat=slice(254, 410), xlon=slice(231, 444))

# 确保掩码对齐
if mask.shape != data_subset.shape:
    # 插值掩码以匹配数据维度
    mask_interp = mask.interp_like(data_subset, method='nearest')
else:
    mask_interp = mask

# 应用掩码
data_masked = data_subset.where(mask_interp, other=-999)

# 保存到 NetCDF 文件
data_masked.to_netcdf(output_file)
print(f'Masked data saved to {output_file}') 

PermissionError: [Errno 13] Permission denied: b'd:\\code\\python\\interpolatedQH_mask1.nc'

In [114]:
mask

<xarray.DataArray 'QH_mask' (y: 156, x: 213)>
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])
Dimensions without coordinates: y, x